[View in Colaboratory](https://colab.research.google.com/github/StefOe/all-conv-pytorch/blob/master/cifar10.ipynb)

In [0]:
cuda = True
train_batch_size = 32
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'

In [0]:
try:
    import torch
except ModuleNotFoundError:
    from os import path
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

    accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.1-{platform}-linux_x86_64.whl
    import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [0]:
try:
    import torchvision
except ModuleNotFoundError:
    !pip install -q torchvision

from torchvision import datasets, transforms

In [0]:
try:
    from allconv import AllConvNet
except ModuleNotFoundError: 
    !wget https://github.com/StefOe/all-conv-pytorch/raw/HEAD/allconv.py
    from allconv import AllConvNet

In [6]:
cuda = cuda and torch.cuda.is_available()
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
train_mean = trainset.train_data.mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
train_std = trainset.train_data.std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
model = AllConvNet(3)
if cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[200, 250, 300], gamma=0.1)

In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
            
def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct /
            len(test_loader.dataset)))
    
    if loss.data[0] < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        model.save("best.pt")

In [0]:
for epoch in range(350):
    scheduler.step()
    train(epoch)
    test(epoch)

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.318764
Train Epoch: 0 [320/50000 (1%)]	Loss: 2.284923
Train Epoch: 0 [640/50000 (1%)]	Loss: 2.292436
Train Epoch: 0 [960/50000 (2%)]	Loss: 2.311650
Train Epoch: 0 [1280/50000 (3%)]	Loss: 2.293617
Train Epoch: 0 [1600/50000 (3%)]	Loss: 2.312588
Train Epoch: 0 [1920/50000 (4%)]	Loss: 2.293349
Train Epoch: 0 [2240/50000 (4%)]	Loss: 2.305540
Train Epoch: 0 [2560/50000 (5%)]	Loss: 2.298046
Train Epoch: 0 [2880/50000 (6%)]	Loss: 2.296811
Train Epoch: 0 [3200/50000 (6%)]	Loss: 2.313303
Train Epoch: 0 [3520/50000 (7%)]	Loss: 2.293413
Train Epoch: 0 [3840/50000 (8%)]	Loss: 2.283757
Train Epoch: 0 [4160/50000 (8%)]	Loss: 2.292461
Train Epoch: 0 [4480/50000 (9%)]	Loss: 2.316245
Train Epoch: 0 [4800/50000 (10%)]	Loss: 2.299819
Train Epoch: 0 [5120/50000 (10%)]	Loss: 2.299110
Train Epoch: 0 [5440/50000 (11%)]	Loss: 2.296137
Train Epoch: 0 [5760/50000 (12%)]	Loss: 2.322785
Train Epoch: 0 [6080/50000 (12%)]	Loss: 2.320367
Train Epoch: 0 [6400/50000 (13%)]	Loss: 2

Train Epoch: 0 [20160/50000 (40%)]	Loss: 2.303979
Train Epoch: 0 [20480/50000 (41%)]	Loss: 2.300681
Train Epoch: 0 [20800/50000 (42%)]	Loss: 2.297668
Train Epoch: 0 [21120/50000 (42%)]	Loss: 2.301889
Train Epoch: 0 [21440/50000 (43%)]	Loss: 2.308232
Train Epoch: 0 [21760/50000 (44%)]	Loss: 2.291711
Train Epoch: 0 [22080/50000 (44%)]	Loss: 2.308837
Train Epoch: 0 [22400/50000 (45%)]	Loss: 2.309948
Train Epoch: 0 [22720/50000 (45%)]	Loss: 2.311386
Train Epoch: 0 [23040/50000 (46%)]	Loss: 2.307814
Train Epoch: 0 [23360/50000 (47%)]	Loss: 2.301917
Train Epoch: 0 [23680/50000 (47%)]	Loss: 2.297677
Train Epoch: 0 [24000/50000 (48%)]	Loss: 2.306062
Train Epoch: 0 [24320/50000 (49%)]	Loss: 2.302821
Train Epoch: 0 [24640/50000 (49%)]	Loss: 2.301436
Train Epoch: 0 [24960/50000 (50%)]	Loss: 2.305444
Train Epoch: 0 [25280/50000 (51%)]	Loss: 2.305934
Train Epoch: 0 [25600/50000 (51%)]	Loss: 2.298136
Train Epoch: 0 [25920/50000 (52%)]	Loss: 2.303322
Train Epoch: 0 [26240/50000 (52%)]	Loss: 2.304988


Train Epoch: 0 [40000/50000 (80%)]	Loss: 2.307673
Train Epoch: 0 [40320/50000 (81%)]	Loss: 2.299012
Train Epoch: 0 [40640/50000 (81%)]	Loss: 2.306967
Train Epoch: 0 [40960/50000 (82%)]	Loss: 2.301695
Train Epoch: 0 [41280/50000 (83%)]	Loss: 2.303169
Train Epoch: 0 [41600/50000 (83%)]	Loss: 2.301627
Train Epoch: 0 [41920/50000 (84%)]	Loss: 2.302511
Train Epoch: 0 [42240/50000 (84%)]	Loss: 2.301566
Train Epoch: 0 [42560/50000 (85%)]	Loss: 2.302002
Train Epoch: 0 [42880/50000 (86%)]	Loss: 2.303195
Train Epoch: 0 [43200/50000 (86%)]	Loss: 2.299551
Train Epoch: 0 [43520/50000 (87%)]	Loss: 2.302970
Train Epoch: 0 [43840/50000 (88%)]	Loss: 2.302221
Train Epoch: 0 [44160/50000 (88%)]	Loss: 2.301253
Train Epoch: 0 [44480/50000 (89%)]	Loss: 2.299019
Train Epoch: 0 [44800/50000 (90%)]	Loss: 2.298443
Train Epoch: 0 [45120/50000 (90%)]	Loss: 2.299006
Train Epoch: 0 [45440/50000 (91%)]	Loss: 2.302169
Train Epoch: 0 [45760/50000 (91%)]	Loss: 2.301176
Train Epoch: 0 [46080/50000 (92%)]	Loss: 2.304559
